## **Binary vs Linear Search**

**Linear search** is the simplest search. It just goes through every single element until finding the target. 

### **Basic Implementation**

In [42]:
# Import relevant modules

from random import randint, sample, random
from typing import TypedDict, Dict, List, Tuple, Optional, Callable, Any
from time import perf_counter_ns as t_ns
from binary_search import binary_search
from collections import defaultdict

In [11]:
# Simple implementation of linear search
def linear_search(ls: list, target: int) -> Optional[int]:
    for i, num in enumerate(ls):
        if num == target:
            return i

    return None

### **Analyzing Complexity**

Binary search halves the search range on every step, whereas linear search only decreases the range by one on every step. So, in theory, binary search should be O(log n), while linear search O(n). In this experiment, I'll set out to showcase this. 

This analysis however, is for the worst case. In practice, one may see other results - such as best case, or, when aggregated, an average case of a given sample. So it is worthwhile thinking of these.

For binary search: 
* **Best case**    - Element is at the middle. So it needs 1 operation.
* **Worst case**   - Element is where the target happens to be the last one checked (or not present). It needs log n.
* **Average case** - Everything else is in between these two. On average, this leads to ~ (log n) - 1. Why?

For linear search:
* **Best case**    - Element is at the beginning. So it needs 1 operation.
* **Worst case**   - Element is at the end (or not present). It needs n operations
* **Average case** - Everything else is in between the two. As it's evenly distributed, this leads to ~ n / 2.

##### **Understanding Average Case of Binary Search**

Although the formal proof can get a bit messy, we can build good intuition for the average case by picturing the sorted list as a complete binary tree representing the comparisons made during binary search.

In such a tree, about half of the elements lie in the bottom level, and the remaining half are distributed across the higher levels.
* Elements in the lowest level require roughly log n comparisons to find.
* The elements in the next level up require about log n − 1 comparisons.

Since each higher level holds only half as many nodes as the level below, most elements are found near the bottom of the tree. After a few levels, you’ve already covered around 75% to 90% of all elements.

Because of that skew, the average depth (and thus the average number of comparisons) ends up being slightly less than log n — roughly around log n − 1.

In short, while the worst case takes about log n comparisons, most searches terminate a little sooner, bringing the average just below that bound.


### **Experimental Set Up**

The base set up will be as follows: 
1. For every size in _sizes_, generate a list as a fixed sorted sequence from 1 to _size_.  
2. Then, select a target and search it with both methods. 
3. Repeat this _attempts_ amount of times.
4. Do this experiment for every hit rate.


##### **Timing the Searches**

In this case, I use a [wrapper function](./wrapper_functions.ipynb) to time the different searches

In [13]:
# Wrapper that returns the time a function takes to run
def timed(func: Callable[..., Any], *args, **kwargs) -> tuple[Any, int]:
    """
    Runs a function and returns (result, elapsed_time_ns).
    """
    start = t_ns()
    result = func(*args, **kwargs)
    elapsed = t_ns() - start
    return result, elapsed


#### **Defining Parameters**

There will be different sizes to measure the impact of a bigger 'n' in the search time, and also an attempt count to get an average and a better approximation of the actual time each takes. Lastly, the experiments will be repeated with different hit rates to also notice behavior at the worst case scenarios.


In [14]:
class SizeResult(TypedDict):                                            # size : {"times": [], "indices": []}
    times:   List[float]
    indices: List[int]

class SearchResults(TypedDict):                                         # search: {size: SizeResult}
    binary:  Dict[int, SizeResult]
    linear:  Dict[int, SizeResult]
    targets: Dict[int, List[int]]

class ExpResults(TypedDict):
    hit_rate: SearchResults

def make_size_result():
    return {"times": [], "indices": []}             

In [ ]:
sizes = [8, 32, 128, 1024, 16384, 131072]                               # sizes = 2**n for n = 3, 5, 7, 10, 17. Each 1 order of magnitude bigger
hit_rates = [0, 0.25, 0.5, 0.75, 1]                                     # ratio of targets inside list (hit) vs outside (miss)
attempts = 100                                                          

##### **Running the Experiment**

In [ ]:
def run_experiment(sizes: List[int], hit_rates: List[int], attempts: int) -> ExpResults:
    exp_results = {}
    for rate in hit_rates:
        results: SearchResults = {
            "binary": defaultdict(make_size_result),
            "linear": defaultdict(make_size_result),
            "targets": defaultdict(list)
        }

        # Run experiment for every size
        for size in sizes:
            nums = list(range(1, size + 1))

            # Check first, middle, last, and a miss.
            indices = [1, size // 2, size - 1, -1]

            # Add a combination of hits and misses in proportion to hit rate
            hits = int(rate*attempts)
            misses = attempts - hits
            if misses: indices.extend([-1]*misses)
            if hits: indices.extend([randint(0, size - 1) for _ in range(hits)])

            # Run and time both searches 'attempts' amount of times
            for idx in indices:
                if idx == -1: target = -1
                else: target = nums[idx]

                lin_i, lin_t = timed(linear_search, nums, target)
                bin_i, bin_t = timed(binary_search, nums, target)

                # Check validity - shouldn't be necessary!
                if lin_i != bin_i:  
                    if idx == -1:   # Expects miss
                        print(f"Expected a miss on both. Got index {lin_i} for linear and {bin_i} for binary.")
                    else:           # Expects hit
                        print(f"Expected index {idx}. Got index {lin_i} for linear and {bin_i} for binary.")

                # Save data
                results["linear"][size]["indices"].append(lin_i)
                results["linear"][size]["times"].append(lin_t)
                results["binary"][size]["indices"].append(bin_i)
                results["binary"][size]["times"].append(bin_t)
                results["targets"][size].append(idx)
            

        exp_results[rate] = results
    
    return exp_results
    


In [26]:
exp_results = run_experiment(sizes, hit_rates, attempts)

### **Visualizing Results**

In [ ]:
import pandas as pd

In [ ]:
# Calculate mean of a data set
def mean(data: List[int]) -> float:
    n = len(data)
    return sum(data) / n

# Calculate trimmed mean - mean without top and bottom 10%
def trimmed(data: List[int], ratio: float) -> float:
    sdata = sorted(data)
    n = len(sdata)
    k = int(n*ratio)
    return mean(sdata[k:n-k])

# Calculate median of a data set 
def median(data: List[int]) -> float:
    sdata = sorted(data)
    n = len(sdata)
    mid = n // 2
    # If it's even, get average of middle 2.
    if n % 2 == 0:
        return (sdata[mid - 1] + sdata[mid]) /  2
    
    return sdata[mid]

# Return values at each given percentile of a data set
def percentiles(data: List[int], percents: List[float]) -> List[float]:
    sdata = sorted(data)
    n = len(sdata) - 1
    results = []
    for p in percents:
        i = int(p*n)
        results.append(sdata[i])
    return results
    

Table with 2 rows, binary and linear. Only use biggest size
Then, 6 columns, search, first, middle, last, miss, average

In [ ]:
size = sizes[-1] # Pick biggest size
results = defaultdict(list)
for search in ["linear", "binary"]:
    times = exp_results[1][search][size]["times"]
    times = [t*10**-3 for t in times] # Convert ns → µs

    results["Search"].append(search)

    # Add individual measurements
    results["First"].append(round(times[0], 2))
    results["Middle"].append(round(times[1], 2))
    results["Last"].append(round(times[2], 2))
    results["Miss"].append(round(times[3], 2))

    # Experimental data
    sample = times[4:] 
    results["Mean"].append(round(mean(sample),2))
    results["T-Mean"].append(round(trimmed(sample, 0.1), 2))
    results["Median"].append(round(median(sample), 2))

    # Get values at each percentile
    percs = {
        0.1: "10th", 0.25: "25th", 0.50: "50th", 0.75: "75th", 
        0.90: "90th", 0.95: "95th", 0.99: "99th"
    }
    perc_values = percentiles(times, list(percs.keys()))
    for label, value in zip(percs.values(), perc_values):
        results[label].append(round(value, 2))
    
df = pd.DataFrame(results)


In [55]:
df

,Search,First,Middle,Last,Miss,Mean,T-Mean,Median,10th,25th,50th,75th,90th,95th,99th
0,linear,1.5,1400.5,2875.3,2861.7,1400.21,1371.38,1267.2,283.3,640.0,1280.2,2176.9,2598.1,2861.7,2952.8
1,binary,4.9,4.4,2.1,2.5,2.35,2.07,1.9,1.5,1.7,1.9,2.5,3.8,4.8,6.5


# Plots!

Plot data - linear vs binary (x-axis = size, y-axis = time). Include different plots for hit rates
Distribution of indices (out of curiosity)
Index vs time for a given size


In [ ]:
# Calculate trimmed mean time on each size
# Plot size vs time, for a given hit rate


In [60]:
res = defaultdict(list)
for rate in hit_rates:
    for size in sizes:
        for search in ["linear", "binary"]:
            times = exp_results[rate][search][size]["times"]
            times = [t*10**-3 for t in times] # Convert ns → µs

            res["id"].append(f"{search}_{size}_{rate}")
            res["Search"].append(search)
            res["Size"].append(size)
            res["Rate"].append(rate)

            # Add individual measurements
            res["First"].append(round(times[0], 2))
            res["Middle"].append(round(times[1], 2))
            res["Last"].append(round(times[2], 2))
            res["Miss"].append(round(times[3], 2))

            # Experimental data
            sample = times[4:] 
            res["Mean"].append(round(mean(sample),2))
            res["T-Mean"].append(round(trimmed(sample, 0.1), 2))
            res["Median"].append(round(median(sample), 2))

            # Get values at each percentile
            percs = {
                0.1: "10th", 0.25: "25th", 0.50: "50th", 0.75: "75th", 
                0.90: "90th", 0.95: "95th", 0.99: "99th"
            }
            perc_values = percentiles(times, list(percs.keys()))
            for label, value in zip(percs.values(), perc_values):
                res[label].append(round(value, 2))
    
df = pd.DataFrame(res)

In [61]:
df

,id,Search,Size,Rate,First,Middle,Last,Miss,Mean,T-Mean,Median,10th,25th,50th,75th,90th,95th,99th
0,linear_8_1,linear,8,1,1.3,0.5,0.4,0.8,0.24,0.24,0.20,0.2,0.2,0.2,0.3,0.3,0.4,0.5
1,binary_8_1,binary,8,1,1.1,0.5,0.4,0.7,0.25,0.26,0.30,0.2,0.2,0.3,0.3,0.3,0.3,0.5
2,linear_32_1,linear,32,1,0.3,0.5,0.8,0.9,0.50,0.43,0.40,0.2,0.3,0.4,0.6,0.7,0.7,0.8
3,binary_32_1,binary,32,1,0.4,0.4,0.4,0.4,0.31,0.32,0.30,0.2,0.3,0.3,0.4,0.4,0.4,0.4
4,linear_128_1,linear,128,1,0.3,1.4,2.5,2.5,1.37,1.38,1.30,0.3,0.9,1.3,2.0,2.3,2.4,2.5
5,binary_128_1,binary,128,1,0.5,0.5,0.5,0.5,0.42,0.43,0.40,0.3,0.4,0.4,0.5,0.5,0.5,0.5
6,linear_1024_1,linear,1024,1,0.2,10.9,20.9,21.5,10.81,10.91,10.75,1.9,6.3,10.8,16.7,19.1,20.0,20.7
7,binary_1024_1,binary,1024,1,1.2,1.5,1.0,1.0,0.73,0.73,0.70,0.6,0.7,0.7,0.8,0.9,1.0,1.2
8,linear_16384_1,linear,16384,1,0.3,175.8,365.3,404.5,167.75,165.44,163.75,37.2,87.7,164.3,244.0,302.6,320.7,404.5
9,binary_16384_1,binary,16384,1,1.1,1.9,1.6,1.5,1.15,1.12,1.10,1.0,1.0,1.1,1.2,1.5,1.6,1.6


In [ ]:
# Plot idx vs frequency

In [ ]:
# Plot idx vs avg time

In [38]:
for size in sizes:
    times_l = exp_results[1]["linear"][size]["times"]
    times_b = exp_results[1]["binary"][size]["times"]

    # Avoid first 4 for this part of the experiment
    avg_b = sum(times_b[4:]) / (attempts * 10**3) # nanosecond to microsecond
    avg_l = sum(times_l[4:]) / (attempts * 10**3)
    print(f"Average time for size {size}: Linear={avg_l: .2f} Binary={avg_b: .2f} [microseconds]")
    print(f"Times for Linear: first={times_l[0]: .2f}, middle={times_l[1]: .2f}, last={times_l[2]: .2f}, miss={times_l[3]: .2f}")
    print(f"Times for Binary: first={times_b[0]: .2f}, middle={times_b[1]: .2f}, last={times_b[2]: .2f}, miss={times_b[3]: .2f}")


Average time for size 8: Linear= 0.24 Binary= 0.25 [microseconds]
Times for Linear: first= 1300.00, middle= 500.00, last= 400.00, miss= 800.00
Times for Binary: first= 1100.00, middle= 500.00, last= 400.00, miss= 700.00
Average time for size 32: Linear= 0.50 Binary= 0.31 [microseconds]
Times for Linear: first= 300.00, middle= 500.00, last= 800.00, miss= 900.00
Times for Binary: first= 400.00, middle= 400.00, last= 400.00, miss= 400.00
Average time for size 128: Linear= 1.37 Binary= 0.42 [microseconds]
Times for Linear: first= 300.00, middle= 1400.00, last= 2500.00, miss= 2500.00
Times for Binary: first= 500.00, middle= 500.00, last= 500.00, miss= 500.00
Average time for size 1024: Linear= 10.81 Binary= 0.73 [microseconds]
Times for Linear: first= 200.00, middle= 10900.00, last= 20900.00, miss= 21500.00
Times for Binary: first= 1200.00, middle= 1500.00, last= 1000.00, miss= 1000.00
Average time for size 16384: Linear= 167.75 Binary= 1.15 [microseconds]
Times for Linear: first= 300.00, m

As expected, binary wins on all but first, as for linear it's the first it checks, while for binary it's among the last.
Also one can notice similar performance for the middle on the first 2 as there's so few elements, they do a similar amount of operations.

In [32]:
from math import log2
for size in sizes:
    print(log2(size))

3.0
5.0
7.0
10.0
14.0
17.0
